# Importy:

In [34]:
# import os
# import json
# import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
# sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
# import torchvision
# from torchvision.utils import save_image
# from torchvision.datasets import FashionMNIST
# from torchvision.transforms import v2
# from torchvision.datasets import ImageFolder
# from torch.nn.functional import one_hot

# from pytorch_fid import fid_score

# from collections import Counter
from sklearn.model_selection import train_test_split

/tmp/ipykernel_7973/294544994.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [35]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cpu


In [36]:
import transformers
from transformers import *
from datasets import load_dataset
import evaluate
from tqdm import tqdm

# Wczytywanie danych

In [37]:
PATH = '../../data'
reviews = pd.read_csv(f"train_data.csv")
reviews.head()

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3


In [38]:
reviews['rating'].value_counts()

rating
4    7243
3    4831
2    1747
1    1434
0    1137
Name: count, dtype: int64

Z komórki wyżej wynika, że dane treningowe są niezbilansowane. \
Znamy 2 główne metody "naprawy" tego problemu: undersampling i oversampling. \
Zdecydowaliśmy się zastosować metodę undersamplingu, w nadziei, że prawie 1200 wierszy dla każdej z klas wystarczy.

In [39]:
X = reviews.filter(['review'])
y = reviews.filter(['rating'])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [41]:
from imblearn.under_sampling import RandomUnderSampler

In [42]:
sampler = RandomUnderSampler(sampling_strategy="not minority")
resampled_data, resampled_classes = sampler.fit_resample(X_train, y_train)

In [43]:
resampled_classes.value_counts()

rating
0         790
1         790
2         790
3         790
4         790
Name: count, dtype: int64

Najpierw podzieliliśmy początkowy zbiór testowy na nowe dane testowe i walidacyjne, a następnie zbilansowaliśmy dane testowe otrzymane w wyniku podziału. Kolejność tych operacji jest ważna, ponieważ chcemy, aby zbiór walidacyjny nie zawierał żadnych zmian, natomiast zbiór testowy został zbilansowany, aby 

In [44]:
resampled = pd.merge(resampled_data, resampled_classes, left_index=True, right_index=True)

In [45]:
resampled.head()

,review,rating
4635,disappointed service there-the restaurant open...,0
13177,"run thought picking cheap hotel save lot, not,...",0
5778,"rating suspect, hotel sofitel june 6-7 power w...",0
3526,"words, ok stay riu hotel punta cana, twice sta...",0
982,n't fooled price location grade room damrak ho...,0


In [46]:
resampled['rating'].value_counts()

rating
0    790
1    790
2    790
3    790
4    790
Name: count, dtype: int64

# Modele

## Model Rekurencyjny

In [21]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)

KeyboardInterrupt: 

In [16]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

In [17]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [18]:
lstm_model = LSTMRegressor(100, 100, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 100)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

## Bag of Words

In [47]:
from bs4 import BeautifulSoup
import re

Aby usprawnić działanie metody BoW, zastosujemy tzw. "stopwords".

In [48]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/michal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:

from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

"Data cleaning"

In [50]:
def review_to_words(raw_review):
    """Function to convert a review to a string of words.
    The input is a single string (a raw moviw review), and the output is a single string (a preprocessed movie review)"""
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))
    meaningful_words = [word for word in words if not word in stops]
    return " ".join(meaningful_words)

In [51]:
i = 0
clean_train_reviews = []
num_reviews = resampled['review'].size

for review in resampled['review']:
    i += 1
    if i % 200 == 0:
        print('Review {} of {}'.format(i, num_reviews))
    clean_train_reviews.append(review_to_words(review))

/tmp/ipykernel_7973/3593606712.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'lxml').get_text()


Review 200 of 3950
Review 400 of 3950
Review 600 of 3950
Review 800 of 3950
Review 1000 of 3950
Review 1200 of 3950
Review 1400 of 3950
Review 1600 of 3950
Review 1800 of 3950
Review 2000 of 3950
Review 2200 of 3950
Review 2400 of 3950
Review 2600 of 3950
Review 2800 of 3950
Review 3000 of 3950
Review 3200 of 3950
Review 3400 of 3950
Review 3600 of 3950
Review 3800 of 3950


In [52]:
print('Creating the bag of words...')
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 1000)
# fit_transform() does two functions: First, it fits the model
# and learns the vocaulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()
print('Bag of words completed')

Creating the bag of words...
Bag of words completed


In [98]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))

1000


In [120]:
flatten_classes = resampled_classes.to_numpy().flatten()

In [121]:
train_indices = np.random.rand(len(train_data_features))>0.3
train_data = torch.from_numpy(train_data_features).float()[train_indices]
train_targets = torch.from_numpy(flatten_classes[train_indices]).long()

test_data = torch.from_numpy(train_data_features[~train_indices]).float()
test_targets = torch.from_numpy(flatten_classes[~train_indices]).long()

In [122]:
train_dataset = torch.utils.data.TensorDataset(train_data, train_targets)
test_dataset = torch.utils.data.TensorDataset(test_data,test_targets)

In [123]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [124]:
class BoWClassifier(nn.Module):
    def __init__(self):
        super(BoWClassifier, self).__init__()
        self.lin1 =nn.Linear(1000, 500)
        self.act1 =nn.LeakyReLU()
        self.lin2 =nn.Linear(500, 50)
        self.act2 =nn.LeakyReLU()
        self.lin3 =nn.Linear(50, 5)


    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x
bow_model = BoWClassifier().to(device)
bow_model

BoWClassifier(
  (lin1): Linear(in_features=1000, out_features=500, bias=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=500, out_features=50, bias=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=50, out_features=5, bias=True)
)

In [125]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for imgs, labels in data_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [127]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bow_model.parameters(), lr=0.001)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(10):
    epoch_losses = []
    for x, labels in iter(train_loader):
        x, labels = x.to(device), labels.to(device)
        bow_model.train()
        out = bow_model(x).squeeze()

        loss = criterion(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(bow_model, test_loader)
    print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
    train_acc.append(get_accuracy(bow_model, train_loader)) # compute training accuracy
    val_acc.append(test_acc)  # compute validation accuracy


print("Final Training Accuracy: {}".format(train_acc[-1]))
print("Final Validation Accuracy: {}".format(val_acc[-1]))

Epoch 0 loss 0.0243 test_acc: 0.486
Epoch 1 loss 0.0107 test_acc: 0.484
Epoch 2 loss 0.00472 test_acc: 0.483
Epoch 3 loss 0.00241 test_acc: 0.483
Epoch 4 loss 0.0015 test_acc: 0.484
Epoch 5 loss 0.0011 test_acc: 0.487
Epoch 6 loss 0.000862 test_acc: 0.487
Epoch 7 loss 0.00069 test_acc: 0.487
Epoch 8 loss 0.000569 test_acc: 0.487
Epoch 9 loss 0.000476 test_acc: 0.485
Final Training Accuracy: 1.0
Final Validation Accuracy: 0.484822934232715


## Gotowe modele z biblioteki sklearn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import FunctionTransformer

In [ ]:
# def np_to_str(X):
#     return X.astype(str)

In [ ]:
pipelineMNB = Pipeline([
    # ('np_to_str', FunctionTransformer(np_to_str)),
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

pipelineCNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ComplementNB())
])

In [ ]:
print(type(resampled_data))
print(type(resampled_classes))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


Debugging

In [ ]:
print(type(resampled_data))
print(type(resampled_classes))
print(resampled_data.shape)
print(resampled_classes.shape)
print(resampled_data.values.reshape(-1))
print(resampled_classes.values.reshape(-1))
print(resampled_data.values.shape)
print(resampled_classes.values.reshape(-1,1).shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(3950, 1)
(3950, 1)
['disappointed service there-the restaurant open breakfast sun-when wanted lunch, not quick place grab bite eat sirens galore idea town location in-and staff extremely rude not helpful, advice-pay little stay better town,  '
 'run thought picking cheap hotel save lot, not, couple staff seen rudeness institutionalized place owner support staff, rooms old worn, bathrooms scarily decrepit tiles chipped, thanks god outside did booked rodent nest- stayed outside time expect time sleep better really exhausted,  '
 'rating suspect, hotel sofitel june 6-7 power water went 24 hours did nothing help 11 oz, bottle water, building effected, did reviews say past, say, letter sofitel admitting problem proof needed glad furnish, saw june 7 th post saying great gives,  '
 ...
 "really enjoyed stay, really loved staying hotel max recent conference, nights thought hotel clean interesting good location downtow

In [ ]:
pipelineMNB.fit(resampled_data.values, resampled_classes.values.reshape(-1, 1))
predict1 = pipelineMNB.predict(X_test)
# print(type(y_test))
# print(type(predict1))
# print(y_test.size)
# print(predict1.size)
print(accuracy_score(y_test, predict1))

AttributeError: 'numpy.ndarray' object has no attribute 'lower'